<a href="https://colab.research.google.com/github/ysnlle/NEURAL-NETWORKS-TECHNOLOGIES-2025-2026/blob/main/mushroom_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение нейронной сети на базе датасета Mushrooms

## Установим все необхоидимое

In [1]:
!pip install kaggle

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("maysee/mushrooms-classification-common-genuss-images")


print("Path to dataset files:", path)

Using Colab cache for faster access to the 'mushrooms-classification-common-genuss-images' dataset.
Path to dataset files: /kaggle/input/mushrooms-classification-common-genuss-images


In [3]:
# указываю полный путь до папок
data_path = f'{path}/Mushrooms'
data_path

'/kaggle/input/mushrooms-classification-common-genuss-images/Mushrooms'

In [4]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Класс датасета

In [7]:
class Mushrooms(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform

        self.data_list = []
        self.len_dataset = 0
        self.classes_count = {0: 0, 1: 0, 2: 0, 3: 0,
                              4: 0, 5: 0, 6: 0, 7: 0, 8: 0}
        self.broken_files = []

        for path_dir, dir_list, file_list in os.walk(path):
            if path_dir == path:
                self.classes = dir_list
                self.class_to_idx = {
                    cls: idx for idx, cls in enumerate(dir_list)
                }
                continue

            cls = path_dir.split('/')[-1]

            for name_file in file_list:
                file_path = os.path.join(path_dir, name_file)

                # здесь, видимо в процессе сжатия, у меня появился поломанный
                # файл, поэтому я решил пожертвовать скоростью, лишь бы избежать
                # поломанных файлов
                try:
                    Image.open(file_path).convert('RGB')
                    self.data_list.append((file_path, self.class_to_idx[cls]))
                    self.classes_count[self.class_to_idx[cls]] += 1
                except Exception as e:
                    print(file_path)
                    self.len_dataset -= 1
                    self.broken_files.append(file_path)

            self.len_dataset += len(file_list)

    def __len__(self):
        return self.len_dataset

    def __getitem__(self, index):
        file_path, target = self.data_list[index]
        sample = Image.open(file_path).convert('RGB')

        if self.transform is not None:
            sample = self.transform(sample)

        return sample, target

In [8]:
# трансформация картинок
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

## Инициализация датасета и разбиение на train/validate/test

In [9]:
# инициализация датасета
data = Mushrooms(data_path, transform)

# выводим вспомогательную инфу
print('length:', len(data))
print('elements at classes: ', data.classes_count)
print('classes to indexces: ', data.class_to_idx)

/kaggle/input/mushrooms-classification-common-genuss-images/Mushrooms/Russula/092_43B354vYxm8.jpg
length: 6713
elements at classes:  {0: 364, 1: 311, 2: 316, 3: 353, 4: 750, 5: 1563, 6: 1147, 7: 1073, 8: 836}
classes to indexces:  {'Entoloma': 0, 'Suillus': 1, 'Hygrocybe': 2, 'Agaricus': 3, 'Amanita': 4, 'Lactarius': 5, 'Russula': 6, 'Boletus': 7, 'Cortinarius': 8}


Разобьем на train/validate/test

In [10]:
# инициализирую размеры частей
train_size = int(0.8 * len(data)) # 0.8
val_size = int(0.1 * len(data)) # 0.1
test_size = len(data) - train_size - val_size # 0.1

# создадим train/validate/split
train_data, val_data, test_data = random_split(data, [train_size, val_size, test_size])

print('train length:', len(train_data))
print('validate length:', len(val_data))
print('test lenght:', len(test_data))

train length: 5370
validate length: 671
test lenght: 672


Классы не сбалансированы, поэтому их надо было забалансить на батчах, чтобы нормально обучить модель

Реализацию подсмотрел [тут](https://traceback.ru/balance-classes-in-torch-with-weighted-random-sampler/). Следующие два блока кода как раз этим и занимался. Также забатчил датасет


In [11]:
from collections import Counter

train_counts = Counter(im[1] for im in train_data)
w = [1 / train_counts[im[1]] for im in train_data]

print(w)
print(train_counts)

[0.0016722408026755853, 0.0035087719298245615, 0.0007974481658692185, 0.0007974481658692185, 0.0036363636363636364, 0.0016722408026755853, 0.001092896174863388, 0.0038461538461538464, 0.001145475372279496, 0.0035087719298245615, 0.001145475372279496, 0.0038461538461538464, 0.0038461538461538464, 0.0007974481658692185, 0.0035087719298245615, 0.0007974481658692185, 0.0007974481658692185, 0.0015220700152207, 0.003952569169960474, 0.001092896174863388, 0.0038461538461538464, 0.0036363636363636364, 0.001145475372279496, 0.001145475372279496, 0.001145475372279496, 0.0016722408026755853, 0.0007974481658692185, 0.003952569169960474, 0.003952569169960474, 0.001145475372279496, 0.001092896174863388, 0.001092896174863388, 0.0016722408026755853, 0.001092896174863388, 0.001145475372279496, 0.0015220700152207, 0.001145475372279496, 0.0015220700152207, 0.001145475372279496, 0.0016722408026755853, 0.0036363636363636364, 0.001145475372279496, 0.001145475372279496, 0.0007974481658692185, 0.0011454753722

In [44]:
from torch.utils.data.sampler import WeightedRandomSampler

sampler = WeightedRandomSampler(w, num_samples=len(w))

train_loader = DataLoader(train_data, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

## Выбор и обучение модели

Решил попробовать поработать с Transfer Learning'ом: заморозил сверточные слои, а линейные создал свои

In [45]:
import torchvision.models as models
from torch.optim import Adam

model = models.resnet50(weights='DEFAULT')

for param in model.parameters(): # замораживаю слои
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Sequential ( # создаю новые линейные слои
    nn.Linear(num_ftrs, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 9)
)
model = model.to(device)

loss_model = nn.CrossEntropyLoss()
optimizer = Adam(model.fc.parameters(), lr=0.001)

In [46]:
from sklearn.metrics import f1_score

EPOCHS = 10

for epoch in range(EPOCHS):

    # Тренировка
    model.train()
    running_train_loss = []
    train_preds = []
    train_targets = []
    train_loop = tqdm(train_loader, leave=False)

    for x, targets in train_loop:
        x = x.to(device)
        targets = targets.to(device)

        pred = model(x)
        loss = loss_model(pred, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss.append(loss.item())
        mean_train_loss = sum(running_train_loss) / len(running_train_loss)

        train_preds.extend(torch.argmax(pred, dim=1).tolist())
        train_targets.extend(targets.tolist())


        train_loop.set_description(f'Epoch[{epoch+1} / {EPOCHS}], train_loss={mean_train_loss:.4f}')

    train_f1 = f1_score(train_targets, train_preds, average='macro')


    # Валидация
    model.eval()
    running_val_loss = []
    val_preds = []
    val_targets = []
    val_loop = tqdm(val_loader, leave=False)

    with torch.no_grad():
        for x, targets in val_loop:
            x = x.to(device)
            targets = targets.to(device)

            pred = model(x)
            loss = loss_model(pred, targets)

            running_val_loss.append(loss.item())
            mean_val_loss = sum(running_val_loss) / len(running_val_loss)

            val_preds.extend(torch.argmax(pred, dim=1).tolist())
            val_targets.extend(targets.tolist())

            val_loop.set_description(f'Epoch[{epoch+1} / {EPOCHS}], val_loss={mean_val_loss:.4f}')

    val_f1 = f1_score(val_targets, val_preds, average='macro')


    print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {mean_train_loss:.4f}, Train F1: {train_f1:.4f}, Val Loss: {mean_val_loss:.4f}, Val F1: {val_f1:.4f}")

Epoch [1/10] Train Loss: 1.5798, Train F1: 0.4736, Val Loss: 1.1812, Val F1: 0.5629


Epoch [2/10] Train Loss: 0.8337, Train F1: 0.7356, Val Loss: 0.9077, Val F1: 0.6789


Epoch [3/10] Train Loss: 0.6012, Train F1: 0.8066, Val Loss: 0.8914, Val F1: 0.6810


Epoch [4/10] Train Loss: 0.5000, Train F1: 0.8420, Val Loss: 0.8789, Val F1: 0.6926


Epoch [5/10] Train Loss: 0.3797, Train F1: 0.8843, Val Loss: 0.7837, Val F1: 0.7071


Epoch [6/10] Train Loss: 0.3346, Train F1: 0.8904, Val Loss: 0.7724, Val F1: 0.7300


Epoch [7/10] Train Loss: 0.2788, Train F1: 0.9096, Val Loss: 0.8961, Val F1: 0.7069


Epoch [8/10] Train Loss: 0.2415, Train F1: 0.9230, Val Loss: 0.8446, Val F1: 0.7171


Epoch [9/10] Train Loss: 0.2293, Train F1: 0.9204, Val Loss: 0.7999, Val F1: 0.7270


Epoch [10/10] Train Loss: 0.1911, Train F1: 0.9375, Val Loss: 0.8350, Val F1: 0.7256


Проверка на тесте

In [47]:
model.eval()
running_test_loss = []
test_preds = []
test_targets = []
test_loop = tqdm(test_loader, leave=False)

with torch.no_grad():
    for x, targets in test_loop:
        x = x.to(device)
        targets = targets.to(device)

        pred = model(x)
        loss = loss_model(pred, targets)

        running_test_loss.append(loss.item())
        mean_test_loss = sum(running_test_loss) / len(running_test_loss)

        test_preds.extend(torch.argmax(pred, dim=1).tolist())
        test_targets.extend(targets.tolist())

        test_loop.set_description(f'Test Loss={mean_test_loss:.4f}')

test_f1 = f1_score(test_targets, test_preds, average='macro')

print(f"Test Loss: {mean_test_loss:.4f}, Test F1: {test_f1:.4f}")

Test Loss: 0.7444, Test F1: 0.7553


# Итог

По метрике F1 получилось 0.75, что, впринципе, не плохо